In [1]:
import os
from dotenv import load_dotenv

# .envファイルを読み込む
load_dotenv()

os.environ["openai_api_key"] = os.getenv("OPENAI_API_KEY")
os.environ["openai_api_base"] = os.getenv("OPENAI_API_BASE")
os.environ["openai_model"] = os.getenv("OPENAI_MODEL")

In [2]:
import sys
import os

# プロジェクトのルートパスを追加（必要に応じて階層を調整）
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

from src.agent import HelpDeskAgent
from src.configs import Settings
from src.tools.search_db import search_db_qa

settings = Settings()


In [3]:
agent = HelpDeskAgent(
    settings=settings,
    #tools=[search_xyz_manual, search_xyz_qa],
    tools=[search_db_qa],
)

In [4]:
# question = """
# お世話になっております。

# 現在、XYZシステムの利用を検討しており、以下の2点についてご教示いただければと存じます。

# 1. パスワードに利用可能な文字の制限について
# 当該システムにてパスワードを設定する際、使用可能な文字の範囲（例：英数字、記号、文字数制限など）について詳しい情報をいただけますでしょうか。安全かつシステムでの認証エラーを防ぐため、具体的な仕様を確認したいと考えております。

# 2. 最新リリースの取得方法について
# 最新のアップデート情報をどのように確認・取得できるかについてもお教えいただけますと幸いです。

# お忙しいところ恐縮ですが、ご対応のほどよろしくお願い申し上げます。
# """
question = """
お世話になっております。

現在Reactの利用を検討しており、以下の2点についてご教示いただければと思います。

1. Reactの初期セットアップについて

2. Reactの主な使い方について

お忙しいところ恐縮ですが、ご対応の程よろしくお願いします。
"""



# question = """
# お世話になっております。

# 現在、XYZシステムを利用を検討しており、以下の点についてご教示いただければと存じます。

# 1. 特定のプロジェクトに対してのみ通知を制限する方法について

# 2. パスワードに利用可能な文字の制限について
# 当該システムにてパスワードを設定する際、使用可能な文字の範囲（例：英数字、記号、文字数制限など）について詳しい情報をいただけますでしょうか。安全かつシステムでの認証エラーを防ぐため、具体的な仕様を確認したいと考えております。

# お忙しいところ恐縮ですが、ご対応のほどよろしくお願い申し上げます。

# """

## 計画ステップ

In [5]:
input_data = {"question": question}

plan_result = agent.create_plan(state=input_data)

2025-07-22 14:33:12,681 INFO 🚀 Starting plan generation process...
2025-07-22 14:33:12,681 INFO Sending request to OpenAI...
2025-07-22 14:33:17,056 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-22 14:33:17,089 INFO ✅ Successfully received response from OpenAI.
2025-07-22 14:33:17,090 INFO Plan generation complete!


In [6]:
plan_result["plan"]

['Reactの初期セットアップに必要なソフトウェアとそのインストール手順について調べる',
 'Reactプロジェクトの作成手順について調べる',
 'Reactの基本的なコンポーネントの作成方法について調べる',
 'Reactの状態管理の基本について調べる',
 'Reactのイベントハンドリングの基本について調べる']

## ツール選択ステップ

In [7]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "is_completed": False,
}

In [8]:
select_tool_result = agent.select_tools(state=input_data)

2025-07-22 14:33:26,639 INFO 🚀 Starting tool selection process...
2025-07-22 14:33:26,642 INFO Sending request to OpenAI...
2025-07-22 14:33:34,955 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-22 14:33:34,964 INFO ✅ Successfully received response from OpenAI.
2025-07-22 14:33:34,971 INFO Tool selection complete!


In [9]:
select_tool_result

{'messages': [{'role': 'system',
   'content': '\nあなたはXYZというシステムの質問応答のためにサブタスク実行を担当するエージェントです。\n回答までの全体の流れは計画立案 → サブタスク実行 [ツール実行 → サブタスク回答 → リフレクション] → 最終回答となります。\nサブタスクはユーザーの質問に回答するために考えられた計画の一つです。\n最終的な回答は全てのサブタスクの結果を組み合わせて別エージェントが作成します。\nあなたは以下の1~3のステップを指示に従ってそれぞれ実行します。各ステップは指示があったら実行し、同時に複数ステップの実行は行わないでください。\nなおリフレクションの結果次第で所定の回数までツール選択・実行を繰り返します。\n\n1. ツール選択・実行\nサブタスク回答のためのツール選択と選択されたツールの実行を行います。\n2回目以降はリフレクションのアドバイスに従って再実行してください。\n\n2. サブタスク回答\nツールの実行結果はあなたしか観測できません。\nツールの実行結果から得られた回答に必要なことは言語化し、最後の回答用エージェントに引き継げるようにしてください。\n例えば、概要を知るサブタスクならば、ツールの実行結果から概要を言語化してください。\n手順を知るサブタスクならば、ツールの実行結果から手順を言語化してください。\n回答できなかった場合は、その旨を言語化してください。\n\n3. リフレクション\nツールの実行結果と回答から、サブタスクに対して正しく回答できているかを評価します。\n回答がわからない、情報が見つからないといった内容の場合は評価をNGにし、やり直すようにしてください。\n評価がNGの場合は、別のツールを試す、別の文言でツールを試すなど、なぜNGなのかとどうしたら改善できるかを考えアドバイスを作成してください。\nアドバイスの内容は過去のアドバイスと計画内の他のサブタスクと重複しないようにしてください。\nアドバイスの内容をもとにツール選択・実行からやり直します。\n評価がOKの場合は、サブタスク回答を終了します。\n\n'},
  {'role': 'user',
   'content': '"\nユーザーの元の質問: \nお世話になっております。

In [10]:
select_tool_result["messages"][-1]

{'role': 'assistant',
 'tool_calls': [{'id': 'call_CTdLwnGKDaTFmZLoT4OnI7yK',
   'function': {'arguments': '{"query":"React 初期セットアップ 必要なソフトウェア インストール手順"}',
    'name': 'search_db_qa'},
   'type': 'function'}]}

In [11]:
select_tool_result["messages"]

[{'role': 'system',
  'content': '\nあなたはXYZというシステムの質問応答のためにサブタスク実行を担当するエージェントです。\n回答までの全体の流れは計画立案 → サブタスク実行 [ツール実行 → サブタスク回答 → リフレクション] → 最終回答となります。\nサブタスクはユーザーの質問に回答するために考えられた計画の一つです。\n最終的な回答は全てのサブタスクの結果を組み合わせて別エージェントが作成します。\nあなたは以下の1~3のステップを指示に従ってそれぞれ実行します。各ステップは指示があったら実行し、同時に複数ステップの実行は行わないでください。\nなおリフレクションの結果次第で所定の回数までツール選択・実行を繰り返します。\n\n1. ツール選択・実行\nサブタスク回答のためのツール選択と選択されたツールの実行を行います。\n2回目以降はリフレクションのアドバイスに従って再実行してください。\n\n2. サブタスク回答\nツールの実行結果はあなたしか観測できません。\nツールの実行結果から得られた回答に必要なことは言語化し、最後の回答用エージェントに引き継げるようにしてください。\n例えば、概要を知るサブタスクならば、ツールの実行結果から概要を言語化してください。\n手順を知るサブタスクならば、ツールの実行結果から手順を言語化してください。\n回答できなかった場合は、その旨を言語化してください。\n\n3. リフレクション\nツールの実行結果と回答から、サブタスクに対して正しく回答できているかを評価します。\n回答がわからない、情報が見つからないといった内容の場合は評価をNGにし、やり直すようにしてください。\n評価がNGの場合は、別のツールを試す、別の文言でツールを試すなど、なぜNGなのかとどうしたら改善できるかを考えアドバイスを作成してください。\nアドバイスの内容は過去のアドバイスと計画内の他のサブタスクと重複しないようにしてください。\nアドバイスの内容をもとにツール選択・実行からやり直します。\n評価がOKの場合は、サブタスク回答を終了します。\n\n'},
 {'role': 'user',
  'content': '"\nユーザーの元の質問: \nお世話になっております。\n\n現在Reactの利用を検

## ツール実行ステップ

In [12]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": select_tool_result["messages"],
    "is_completed": False,
}

In [13]:
tool_results = agent.execute_tools(state=input_data)

2025-07-22 14:33:47,660 INFO 🚀 Starting tool execution process...
2025-07-22 14:33:47,668 INFO Searching by query: {"query":"React 初期セットアップ 必要なソフトウェア インストール手順"}
2025-07-22 14:33:48,290 INFO Generating embedding vector from input query
2025-07-22 14:33:49,026 INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[(4, 'React ナレッジ\nオーナー\n福田拓哉\nタグ\n手順 / プロセス\nプロジェクトの開始の仕方\n前提：Node.jsがインストールされていること\nhttps://nodejs.org/ja/download\nプロジェクトの開始\n下記はReact×Typescript\nnpx create-react-app {プロジェクト名} --template typescript\n最近はこっちになったらしい\nhttps://qiita.com/gota5010/items/0caf9cf519bed85ea1fc\nnpm create vite@latest\ncd プロジェクト名\nnpm install', {'source': 'PDF', 'file_name': '4615daec-bd30-4bce-a5ff-9fdac962a732_React_ナレッジ.pdf'}), (5, 'https://qiita.com/gota5010/items/0caf9cf519bed85ea1fc\nnpm create vite@latest\ncd プロジェクト名\nnpm install\nnpm run dev\nReactプロジェクトのバージョン確認コマンド\nReact ナレッジ\n1\nnpm list react\nレンダリングとは\n再描画すること。ブラウザで画面リロードするわけではないけれど、必要なUIの部\n分だけを"もう一度作って表示し直す\n再レンダ

In [14]:
tool_results["tool_results"][0][0].results

[SearchOutput(file_name='4615daec-bd30-4bce-a5ff-9fdac962a732_React_ナレッジ.pdf', content='React ナレッジ\nオーナー\n福田拓哉\nタグ\n手順 / プロセス\nプロジェクトの開始の仕方\n前提：Node.jsがインストールされていること\nhttps://nodejs.org/ja/download\nプロジェクトの開始\n下記はReact×Typescript\nnpx create-react-app {プロジェクト名} --template typescript\n最近はこっちになったらしい\nhttps://qiita.com/gota5010/items/0caf9cf519bed85ea1fc\nnpm create vite@latest\ncd プロジェクト名\nnpm install'),
 SearchOutput(file_name='4615daec-bd30-4bce-a5ff-9fdac962a732_React_ナレッジ.pdf', content='https://qiita.com/gota5010/items/0caf9cf519bed85ea1fc\nnpm create vite@latest\ncd プロジェクト名\nnpm install\nnpm run dev\nReactプロジェクトのバージョン確認コマンド\nReact ナレッジ\n1\nnpm list react\nレンダリングとは\n再描画すること。ブラウザで画面リロードするわけではないけれど、必要なUIの部\n分だけを"もう一度作って表示し直す\n再レンダリングされるとき\nステートが変わる時\n開発モードの時は勝手に２回レンダリングされる'),
 SearchOutput(file_name='4615daec-bd30-4bce-a5ff-9fdac962a732_React_ナレッジ.pdf', content='</StrictMode>\n);\n開発の時はStrictModeがついているので2回レンダリングされるのは普通。\n通常npm run buildすると自動的に削除してくれる\nReactのif文的なところ\nあんまりif文は使わず下記の書き方が

In [15]:
tool_results

{'messages': [{'role': 'system',
   'content': '\nあなたはXYZというシステムの質問応答のためにサブタスク実行を担当するエージェントです。\n回答までの全体の流れは計画立案 → サブタスク実行 [ツール実行 → サブタスク回答 → リフレクション] → 最終回答となります。\nサブタスクはユーザーの質問に回答するために考えられた計画の一つです。\n最終的な回答は全てのサブタスクの結果を組み合わせて別エージェントが作成します。\nあなたは以下の1~3のステップを指示に従ってそれぞれ実行します。各ステップは指示があったら実行し、同時に複数ステップの実行は行わないでください。\nなおリフレクションの結果次第で所定の回数までツール選択・実行を繰り返します。\n\n1. ツール選択・実行\nサブタスク回答のためのツール選択と選択されたツールの実行を行います。\n2回目以降はリフレクションのアドバイスに従って再実行してください。\n\n2. サブタスク回答\nツールの実行結果はあなたしか観測できません。\nツールの実行結果から得られた回答に必要なことは言語化し、最後の回答用エージェントに引き継げるようにしてください。\n例えば、概要を知るサブタスクならば、ツールの実行結果から概要を言語化してください。\n手順を知るサブタスクならば、ツールの実行結果から手順を言語化してください。\n回答できなかった場合は、その旨を言語化してください。\n\n3. リフレクション\nツールの実行結果と回答から、サブタスクに対して正しく回答できているかを評価します。\n回答がわからない、情報が見つからないといった内容の場合は評価をNGにし、やり直すようにしてください。\n評価がNGの場合は、別のツールを試す、別の文言でツールを試すなど、なぜNGなのかとどうしたら改善できるかを考えアドバイスを作成してください。\nアドバイスの内容は過去のアドバイスと計画内の他のサブタスクと重複しないようにしてください。\nアドバイスの内容をもとにツール選択・実行からやり直します。\n評価がOKの場合は、サブタスク回答を終了します。\n\n'},
  {'role': 'user',
   'content': '"\nユーザーの元の質問: \nお世話になっております。

## サブタスク回答

In [16]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": tool_results["messages"],
    "tool_results": tool_results["tool_results"],
    "is_completed": False,
}

In [17]:
subtask_answer = agent.create_subtask_answer(state=input_data)

2025-07-22 14:33:56,873 INFO 🚀 Starting subtask answer creation process...
2025-07-22 14:33:56,873 INFO Sending request to OpenAI...
2025-07-22 14:34:01,820 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-22 14:34:01,831 INFO ✅ Successfully received response from OpenAI.
2025-07-22 14:34:01,832 INFO Subtask answer creation complete!


In [18]:
subtask_answer

{'messages': [{'role': 'system',
   'content': '\nあなたはXYZというシステムの質問応答のためにサブタスク実行を担当するエージェントです。\n回答までの全体の流れは計画立案 → サブタスク実行 [ツール実行 → サブタスク回答 → リフレクション] → 最終回答となります。\nサブタスクはユーザーの質問に回答するために考えられた計画の一つです。\n最終的な回答は全てのサブタスクの結果を組み合わせて別エージェントが作成します。\nあなたは以下の1~3のステップを指示に従ってそれぞれ実行します。各ステップは指示があったら実行し、同時に複数ステップの実行は行わないでください。\nなおリフレクションの結果次第で所定の回数までツール選択・実行を繰り返します。\n\n1. ツール選択・実行\nサブタスク回答のためのツール選択と選択されたツールの実行を行います。\n2回目以降はリフレクションのアドバイスに従って再実行してください。\n\n2. サブタスク回答\nツールの実行結果はあなたしか観測できません。\nツールの実行結果から得られた回答に必要なことは言語化し、最後の回答用エージェントに引き継げるようにしてください。\n例えば、概要を知るサブタスクならば、ツールの実行結果から概要を言語化してください。\n手順を知るサブタスクならば、ツールの実行結果から手順を言語化してください。\n回答できなかった場合は、その旨を言語化してください。\n\n3. リフレクション\nツールの実行結果と回答から、サブタスクに対して正しく回答できているかを評価します。\n回答がわからない、情報が見つからないといった内容の場合は評価をNGにし、やり直すようにしてください。\n評価がNGの場合は、別のツールを試す、別の文言でツールを試すなど、なぜNGなのかとどうしたら改善できるかを考えアドバイスを作成してください。\nアドバイスの内容は過去のアドバイスと計画内の他のサブタスクと重複しないようにしてください。\nアドバイスの内容をもとにツール選択・実行からやり直します。\n評価がOKの場合は、サブタスク回答を終了します。\n\n'},
  {'role': 'user',
   'content': '"\nユーザーの元の質問: \nお世話になっております。

In [19]:
print(subtask_answer["subtask_answer"])

Reactの初期セットアップに必要なソフトウェアとそのインストール手順については以下の通りです。

1. **Node.jsのインストール**:
   - Reactのプロジェクトを開始するためには、まずNode.jsがインストールされている必要があります。
   - Node.jsの公式サイト（https://nodejs.org/ja/download）からダウンロードしてインストールしてください。

2. **Reactプロジェクトの作成**:
   - Node.jsがインストールされたら、Reactプロジェクトを作成するために以下のコマンドを使用します。
   - `npx create-react-app {プロジェクト名} --template typescript` これはTypeScriptを使用したReactプロジェクトの作成方法です。
   - 最近では、`npm create vite@latest`を使用してプロジェクトを作成する方法もあります。
   - プロジェクトディレクトリに移動し、`npm install`を実行して依存関係をインストールします。

これらの手順に従うことで、Reactの初期セットアップを行うことができます。


## リフレクション

In [20]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": subtask_answer["messages"],
    "tool_results": tool_results["tool_results"],
    "is_completed": False,
    "subtask_answer": subtask_answer["subtask_answer"],
}

In [21]:
reflection_result = agent.reflect_subtask(state=input_data)

2025-07-22 14:34:18,807 INFO 🚀 Starting reflection process...
2025-07-22 14:34:18,808 INFO Sending request to OpenAI...
2025-07-22 14:34:21,453 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-22 14:34:21,476 INFO ✅ Successfully received response from OpenAI.
2025-07-22 14:34:21,477 INFO Reflection complete!


In [22]:
reflection_result

{'messages': [{'role': 'system',
   'content': '\nあなたはXYZというシステムの質問応答のためにサブタスク実行を担当するエージェントです。\n回答までの全体の流れは計画立案 → サブタスク実行 [ツール実行 → サブタスク回答 → リフレクション] → 最終回答となります。\nサブタスクはユーザーの質問に回答するために考えられた計画の一つです。\n最終的な回答は全てのサブタスクの結果を組み合わせて別エージェントが作成します。\nあなたは以下の1~3のステップを指示に従ってそれぞれ実行します。各ステップは指示があったら実行し、同時に複数ステップの実行は行わないでください。\nなおリフレクションの結果次第で所定の回数までツール選択・実行を繰り返します。\n\n1. ツール選択・実行\nサブタスク回答のためのツール選択と選択されたツールの実行を行います。\n2回目以降はリフレクションのアドバイスに従って再実行してください。\n\n2. サブタスク回答\nツールの実行結果はあなたしか観測できません。\nツールの実行結果から得られた回答に必要なことは言語化し、最後の回答用エージェントに引き継げるようにしてください。\n例えば、概要を知るサブタスクならば、ツールの実行結果から概要を言語化してください。\n手順を知るサブタスクならば、ツールの実行結果から手順を言語化してください。\n回答できなかった場合は、その旨を言語化してください。\n\n3. リフレクション\nツールの実行結果と回答から、サブタスクに対して正しく回答できているかを評価します。\n回答がわからない、情報が見つからないといった内容の場合は評価をNGにし、やり直すようにしてください。\n評価がNGの場合は、別のツールを試す、別の文言でツールを試すなど、なぜNGなのかとどうしたら改善できるかを考えアドバイスを作成してください。\nアドバイスの内容は過去のアドバイスと計画内の他のサブタスクと重複しないようにしてください。\nアドバイスの内容をもとにツール選択・実行からやり直します。\n評価がOKの場合は、サブタスク回答を終了します。\n\n'},
  {'role': 'user',
   'content': '"\nユーザーの元の質問: \nお世話になっております。

In [23]:
# 最初に選択されたツールを確認
print(reflection_result["messages"][2]["tool_calls"][0]["function"]["name"])

search_db_qa


In [24]:
# リフレクション結果の確認
print("is_completed =", reflection_result["reflection_results"][0].is_completed)
print("advice =", reflection_result["reflection_results"][0].advice)

is_completed = True
advice = サブタスクの内容に対して、Node.jsのインストールとReactプロジェクトの作成手順を正しく提供できています。特に、TypeScriptを使用したプロジェクトの作成方法と、Viteを使用した最新のプロジェクト作成方法についても言及しており、情報は十分です。したがって、評価はOKです。
